In [1]:
import xarray as xr 
import matplotlib.pyplot as plt 
import glob 
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import matplotlib.ticker as mticker
from matplotlib import ticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cmocean
import scipy as sp
from scipy.stats import linregress
import sklearn.linear_model as LR
import scipy.stats as stats
import cmasher as cmr
import geocat.comp as gcomp

import pandas as pd
from dask.distributed import get_client, Client
import time
import os

In [2]:
# Get the current client if one exists and close it
try:
    client = get_client()
    client.close()
except ValueError:
    print("No active client found.")

client = Client()
print(f"Dask dashboard available at: {client.dashboard_link}")
time.sleep(19)  # Sleeps for 1 second

No active client found.
Dask dashboard available at: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Grow/proxy/8787/status


# Prep any climo variables eaten by the scratch monster:

In [80]:
ls /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/

CAM5_Betzy1/  FHIST5_f09_g16_v2/  MME_Combined/
CAM6_Betzy1/  FHIST_f09_g16/      MME_Regular/


In [82]:
top_dir = '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/'
yrs = '197901-200512'
time_range = ('1979-01-01', '2005-12-01')

all_vars = ['U','V','T','Q','PSL','Z3']
all_mods = ['MME_Regular','FHIST5_f09_g16_v2','CAM5_Betzy1','CAM6_Betzy1']

for mname in all_mods:
    for var_name in all_vars:
        print('doing :', var_name)
        full_dir_ts = f'{top_dir}/{mname}/ts/{mname}.cam.h0.{var_name}.{yrs}.nc'
        DSall = xr.open_dataset(full_dir_ts).drop_vars('time_bnds')
        DSall
        
         # # # Assign proper time coordinate
        dates = pd.date_range(*time_range, freq='1MS')
        dates_to_remove = [] #['1986-05-01']
        filtered_dates = dates[~dates.isin(pd.to_datetime(dates_to_remove))]
        DSall['time'] = filtered_dates
        
        # Compute monthly climatology efficiently
        monthly_climo = DSall.groupby("time.month").mean("time").compute()
        
        # Rename `month` to `time`
        monthly_climo = monthly_climo.rename({'month': 'time'})
        
        # Save to NetCDF
        os.makedirs(f'/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/{mname}/climo_rv/', exist_ok=True)
        output_file = f'/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/{mname}/climo_rv/{mname}_{var_name}_climo.nc'
        monthly_climo.to_netcdf(output_file)
        
        print(f"Saved: {output_file}")

doing : Z3
Saved: /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/MME_Regular/climo_rv/MME_Regular_Z3_climo.nc
doing : Z3
Saved: /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/FHIST5_f09_g16_v2/climo_rv/FHIST5_f09_g16_v2_Z3_climo.nc
doing : Z3
Saved: /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM5_Betzy1/climo_rv/CAM5_Betzy1_Z3_climo.nc
doing : Z3
Saved: /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM6_Betzy1/climo_rv/CAM6_Betzy1_Z3_climo.nc


## Set all of your Functions and Settings:

In [2]:
SEAS = 'ANN'

seasons =  {"ANN": np.arange(1,13,1),
            "DJF": [12, 1, 2],
            "JJA": [6, 7, 8],
            "MAM": [3, 4, 5],
            "SON": [9, 10, 11]
            }
DTDT = pd.date_range(start='1980-01-01',end='1980-12-01',freq='1MS')
# Create a DataFrame to hold the date range
df = pd.DataFrame({'date': DTDT})

# Extract the number of days in each month
df['days_in_month'] = df['date'].dt.days_in_month
df['month'] = df['date'].dt.month
df

,date,days_in_month,month
0,1980-01-01,31,1
1,1980-02-01,29,2
2,1980-03-01,31,3
3,1980-04-01,30,4
4,1980-05-01,31,5
5,1980-06-01,30,6
6,1980-07-01,31,7
7,1980-08-01,31,8
8,1980-09-01,30,9
9,1980-10-01,31,10


In [12]:
def seas_mean(DS, df, season):
    DSdo = DS.sel(time=DS.time.isin(seasons[season]))

    if np.isin(season,['DJF','MAM','SON','JJA']):
        m1w=df[df['month']==DSdo.time[0].values]['days_in_month'].values[0]
        m2w=df[df['month']==DSdo.time[1].values]['days_in_month'].values[0]
        m3w=df[df['month']==DSdo.time[2].values]['days_in_month'].values[0]
        weights = np.array([m1w,m2w,m3w])
    if season =='ANN':
        m1w=df[df['month']==DSdo.time[0].values]['days_in_month'].values[0]
        m2w=df[df['month']==DSdo.time[1].values]['days_in_month'].values[0]
        m3w=df[df['month']==DSdo.time[2].values]['days_in_month'].values[0]
        m4w=df[df['month']==DSdo.time[3].values]['days_in_month'].values[0]
        m5w=df[df['month']==DSdo.time[4].values]['days_in_month'].values[0]
        m6w=df[df['month']==DSdo.time[5].values]['days_in_month'].values[0]
        m7w=df[df['month']==DSdo.time[6].values]['days_in_month'].values[0]
        m8w=df[df['month']==DSdo.time[7].values]['days_in_month'].values[0]
        m9w=df[df['month']==DSdo.time[8].values]['days_in_month'].values[0]
        m10w=df[df['month']==DSdo.time[9].values]['days_in_month'].values[0]
        m11w=df[df['month']==DSdo.time[10].values]['days_in_month'].values[0]
        m12w=df[df['month']==DSdo.time[11].values]['days_in_month'].values[0]
        weights = np.array([m1w,m2w,m3w,m4w,m5w,m6w,m7w,m8w,m9w,m10w,m11w,m12w])
        
    weights = weights/weights.sum()
    ds_weighted = (DSdo*np.expand_dims(np.expand_dims(weights,1),1))
    DSseas = ds_weighted.sum(dim='time')

    return DSseas.load()

def wgt_rmse(fld1, fld2, wgt):
    """Calculate the area-weighted RMSE.

    Parameters
    ----------
    fld1, fld2 : array-like
        2-dimensional spatial fields with the same shape.
        They can be xarray DataArray or numpy arrays.
    wgt : array-like
        the weight vector, expected to be 1-dimensional,
        matching length of one dimension of the data.

    Returns
    -------
    float
        root mean squared error

    Notes:
    ```rmse = sqrt( mean( (fld1 - fld2)**2 ) )```
    """
    assert len(fld1.shape) == 2,     "Input fields must have exactly two dimensions."
    assert fld1.shape == fld2.shape, "Input fields must have the same array shape."
    # in case these fields are in dask arrays, compute them now.
    if hasattr(fld1, "compute"):
        fld1 = fld1.compute()
    if hasattr(fld2, "compute"):
        fld2 = fld2.compute()
    if isinstance(fld1, xr.DataArray) and isinstance(fld2, xr.DataArray):
        return (np.sqrt(((fld1 - fld2)**2).weighted(wgt).mean())).values.item()
    else:
        check = [len(wgt) == s for s in fld1.shape]
        if ~np.any(check):
            raise IOError(f"Sorry, weight array has shape {wgt.shape} which is not compatible with data of shape {fld1.shape}")
        check = [len(wgt) != s for s in fld1.shape]
        dimsize = fld1.shape[np.argwhere(check).item()]  # want to get the dimension length for the dim that does not match the size of wgt
        warray = np.tile(wgt, (dimsize, 1)).transpose()   # May need more logic to ensure shape is correct.
        warray = warray / np.sum(warray) # normalize
        wmse = np.sum(warray * (fld1 - fld2)**2)
        return np.sqrt( wmse ).item()

def process_model(model_name, file_path, levs_interp, df, SEAS, var):
    """Generalized function to process climate model data and compute RMSE."""
    
    # Load datasets dynamically based on model_name
    ds_mlev = xr.open_dataset(f'{file_path}/{model_name}_{var}_climo.nc')
    ds_mlev_z = xr.open_dataset(f'{file_path}/{model_name}_Z3_climo.nc')
    ds_mlev_tbot = xr.open_dataset(f'{file_path}/{model_name}_T_climo.nc')
    
    # Extract latitude and weights
    lat = ds_mlev['lat']
    wgt = np.cos(np.radians(lat))

    # Load observational data and take mean over time
    # Get hybrid pressure coefficients
    hyam = ds_mlev['hyam']
    hybm = ds_mlev['hybm']

    # Interpolation to pressure levels
    inter_data = gcomp.interpolation.interp_hybrid_to_pressure(
        data=ds_mlev[var], ps=ds_mlev['PS'], hyam=hyam, hybm=hybm, 
        new_levels=levs_interp * 100, extrapolate=True, variable='other',
        t_bot=ds_mlev_tbot['T'].sel(lev=1000, method='nearest'), 
        phi_sfc=ds_mlev_z['Z3']
    ).load().to_dataset(name=var)

    # Convert pressure levels to hPa and rename
    inter_data['plev'] = inter_data['plev'] / 100
    inter_data = inter_data.rename({'plev': 'lev'})

    return inter_data

def process_model_flat(model_name, file_path, levs_interp, df, SEAS, var):
    """Generalized function to process climate model data and compute RMSE."""
    
    # Load datasets dynamically based on model_name
    ds_mlev = xr.open_dataset(f'{file_path}/{model_name}_{var}_climo.nc')
    return ds_mlev

In [13]:
ls /glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/

CAM5_Betzy1/  FHIST5_f09_g16_v2/  MME_Combined/
CAM6_Betzy1/  FHIST_f09_g16/      MME_Regular/


In [16]:
# # Example usage

vardo = ['PSL','U','V','T','Q']
e5v = ['MSL','u','v','T','Q']

ffil = '/glade/campaign/cgd/amp/wchapman/Reanalysis/'

e5f = [f'{ffil}/e5.moda.an.sfc.128_151_msl.ll025sc.camgrid.1979.2022.nc',
       f'{ffil}/e5.moda.an.pl.128_131_u.ll025uv.camgrid.1979.2022.nc',
       f'{ffil}/e5.moda.an.pl.128_131_v.ll025uv.camgrid.1979.2022.nc',
       f'{ffil}/e5.moda.an.pl.128_130_t.ll025sc.camgrid.1979.2022.nc',
       f'{ffil}/e5.moda.an.pl.128_131_q.ll025sc.camgrid.1979.2022.nc']

for ee,vvv in enumerate(vardo):
    DS_mme_mlev = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/MME_Regular/climo/MME_Regular_U_climo.nc')
    lat = DS_mme_mlev['lat']
    wgt = np.cos(np.radians(lat))
    
    levs_interp = np.array([100., 125., 150., 175., 200., 225., 250., 300., 350., 400.,
                             450., 500., 550., 600., 650., 700., 750., 775., 800., 825.,
                             850., 875., 900., 925., 950., 975., 1000.])


    if vvv=='PSL':
        
        INTER_mme = process_model_flat('MME_Regular', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/MME_Regular/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_super5 = process_model_flat('CAM5_Betzy1', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM5_Betzy1/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_super6 = process_model_flat('CAM6_Betzy1', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM6_Betzy1/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_cam6 = process_model_flat('FHIST_f09_g16', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/FHIST_f09_g16/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_MMEr = process_model_flat('MME_Combined', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/MME_Combined/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_cam5 = process_model_flat('FHIST5_f09_g16_v2', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/FHIST5_f09_g16_v2/climo_rv/', levs_interp, df, SEAS, var=vvv)
    else:
        INTER_mme = process_model('MME_Regular', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/MME_Regular/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_super5 = process_model('CAM5_Betzy1', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM5_Betzy1/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_super6 = process_model('CAM6_Betzy1', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/CAM6_Betzy1/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_cam6 = process_model('FHIST_f09_g16', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/FHIST_f09_g16/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_MMEr = process_model('MME_Combined', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/MME_Combined/climo_rv/', levs_interp, df, SEAS, var=vvv)
        INTER_cam5 = process_model('FHIST5_f09_g16_v2', '/glade/campaign/cgd/amp/wchapman/ADF/SUMO_REVIEW_02/FHIST5_f09_g16_v2/climo_rv/', levs_interp, df, SEAS, var=vvv)
    
    print('taking obs mean')
    DS_obs = xr.open_dataset(e5f[ee]).mean('time')
    print('taking obs mean')

    if vvv=='PSL':
        INTER_mme_season = seas_mean(INTER_mme.squeeze(), df, SEAS)
        mme_rmse = wgt_rmse(INTER_mme_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
    
        INTER_MMEr_season = seas_mean(INTER_MMEr.squeeze(), df, SEAS)
        MMEr_rmse = wgt_rmse(INTER_MMEr_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
    
        INTER_super5_season = seas_mean(INTER_super5.squeeze(), df, SEAS)
        super5_rmse = wgt_rmse(INTER_super5_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
         
        INTER_super6_season = seas_mean(INTER_super6.squeeze(), df, SEAS)
        super6_rmse = wgt_rmse(INTER_super6_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
    
        INTER_cam5_season = seas_mean(INTER_cam5.squeeze(), df, SEAS)
        cam5_rmse = wgt_rmse(INTER_cam5_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
         
        INTER_cam6_season = seas_mean(INTER_cam6.squeeze(), df, SEAS)
        cam6_rmse = wgt_rmse(INTER_cam6_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]],wgt.sel(lat=slice(-90,90)))
         
        print(f'var: {vvv}; RMSE NIense: {mme_rmse}, SUPERense: {MMEr_rmse}, SUMO5: {super5_rmse}, SUMO6: {super6_rmse}, CAM5: {cam5_rmse}, CAM6: {cam6_rmse}' )
        
    else:
        for levlev in levs_interp:
            INTER_mme_season = seas_mean(INTER_mme.sel(lev=levlev).squeeze(), df, SEAS)
            mme_rmse = wgt_rmse(INTER_mme_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
        
            INTER_MMEr_season = seas_mean(INTER_MMEr.sel(lev=levlev).squeeze(), df, SEAS)
            MMEr_rmse = wgt_rmse(INTER_MMEr_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
        
            INTER_super5_season = seas_mean(INTER_super5.sel(lev=levlev).squeeze(), df, SEAS)
            super5_rmse = wgt_rmse(INTER_super5_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
             
            INTER_super6_season = seas_mean(INTER_super6.sel(lev=levlev).squeeze(), df, SEAS)
            super6_rmse = wgt_rmse(INTER_super6_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
        
            INTER_cam5_season = seas_mean(INTER_cam5.sel(lev=levlev).squeeze(), df, SEAS)
            cam5_rmse = wgt_rmse(INTER_cam5_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
             
            INTER_cam6_season = seas_mean(INTER_cam6.sel(lev=levlev).squeeze(), df, SEAS)
            cam6_rmse = wgt_rmse(INTER_cam6_season.sel(lat=slice(-90,90))[vvv],DS_obs.sel(lat=slice(-90,90))[e5v[ee]].sel(level=levlev),wgt.sel(lat=slice(-90,90)))
             
            print(f'var: {vvv}; Level {levlev}mb; RMSE NIense: {mme_rmse}, SUPERense: {MMEr_rmse}, SUMO5: {super5_rmse}, SUMO6: {super6_rmse}, CAM5: {cam5_rmse}, CAM6: {cam6_rmse}' )

taking obs mean
taking obs mean
var: PSL; RMSE NIense: 163.08395360599874, SUPERense: 190.60824601885398, SUMO5: 182.0794291590091, SUMO6: 206.02842357052276, CAM5: 144.71210930489192, CAM6: 224.21111284956288


/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.1

taking obs mean
taking obs mean
var: U; Level 100.0mb; RMSE NIense: 2.0332707687140723, SUPERense: 2.2251690219399567, SUMO5: 2.308166091948088, SUMO6: 2.187084019258799, CAM5: 2.485052090247107, CAM6: 2.6526522350100237
var: U; Level 125.0mb; RMSE NIense: 2.0175091265201974, SUPERense: 2.2017841323405105, SUMO5: 2.2689728024502283, SUMO6: 2.1991403744031035, CAM5: 2.499006332609046, CAM6: 2.80029082665264
var: U; Level 150.0mb; RMSE NIense: 1.8980443524690878, SUPERense: 2.1578750069350794, SUMO5: 2.1621994837620826, SUMO6: 2.2294003908441296, CAM5: 2.253201839134047, CAM6: 2.7087410063676787
var: U; Level 175.0mb; RMSE NIense: 1.8305753803686502, SUPERense: 2.1159031277447653, SUMO5: 2.0777454339860113, SUMO6: 2.229757745213301, CAM5: 2.04665916678503, CAM6: 2.58255364759104
var: U; Level 200.0mb; RMSE NIense: 1.7365171759902596, SUPERense: 2.019605778117204, SUMO5: 1.956224670894599, SUMO6: 2.1528518342553795, CAM5: 1.880222915620662, CAM6: 2.4179630008519757
var: U; Level 225.0mb; 

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.1

taking obs mean
taking obs mean
var: V; Level 100.0mb; RMSE NIense: 0.5812718737440769, SUPERense: 0.6079473762964247, SUMO5: 0.6163089048612989, SUMO6: 0.6289655663384602, CAM5: 0.6303651364537391, CAM6: 0.734087542066567
var: V; Level 125.0mb; RMSE NIense: 0.7185298212088578, SUPERense: 0.7473566535559637, SUMO5: 0.7527989398050974, SUMO6: 0.7925153448679132, CAM5: 0.7993745953437041, CAM6: 0.9034636563769868
var: V; Level 150.0mb; RMSE NIense: 0.7848007002600553, SUPERense: 0.856807919196637, SUMO5: 0.8508182356656532, SUMO6: 0.9346043107823531, CAM5: 0.8464415310162153, CAM6: 1.0241943644126998
var: V; Level 175.0mb; RMSE NIense: 0.8223049021821943, SUPERense: 0.9252034797841229, SUMO5: 0.9023000514652094, SUMO6: 1.0248484789431582, CAM5: 0.8543994756479834, CAM6: 1.0936684574321207
var: V; Level 200.0mb; RMSE NIense: 0.8500523549792987, SUPERense: 0.9529244707332951, SUMO5: 0.9196315433465291, SUMO6: 1.0565882081913298, CAM5: 0.87501757215418, CAM6: 1.1118473622566856
var: V; Leve

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.1

taking obs mean
taking obs mean
var: T; Level 100.0mb; RMSE NIense: 0.8284884542136268, SUPERense: 1.0137372715409017, SUMO5: 1.0636434048069268, SUMO6: 0.980014556121862, CAM5: 1.1008192146508782, CAM6: 0.9169546662508546
var: T; Level 125.0mb; RMSE NIense: 1.0189025770899425, SUPERense: 1.3478468629310894, SUMO5: 1.4018600002910895, SUMO6: 1.300681547772378, CAM5: 1.2905094530337273, CAM6: 0.9448280450372873
var: T; Level 150.0mb; RMSE NIense: 1.4986754117570362, SUPERense: 1.8855251682839431, SUMO5: 1.9343395981571285, SUMO6: 1.8396549659272485, CAM5: 1.937190720059591, CAM6: 1.2161259619126725
var: T; Level 175.0mb; RMSE NIense: 1.9117239928444978, SUPERense: 2.288584510327957, SUMO5: 2.331455137729936, SUMO6: 2.2499844804466846, CAM5: 2.4416874914322806, CAM6: 1.5336953738156924
var: T; Level 200.0mb; RMSE NIense: 2.0525686935486043, SUPERense: 2.3689959542940735, SUMO5: 2.4078308006497156, SUMO6: 2.3381163023929106, CAM5: 2.552485424854608, CAM6: 1.7067838121176555
var: T; Level 

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/geocat/comp/interpolation.py:133: UserWarning: Interpolation point out of data bounds encountered
  return func_interpolate(new_levels, xcoords, data, axis=interp_axis)
/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.1

taking obs mean
taking obs mean
var: Q; Level 100.0mb; RMSE NIense: 3.236870563636293e-07, SUPERense: 2.9366070792695974e-07, SUMO5: 2.494842184009475e-07, SUMO6: 3.5617845000742794e-07, CAM5: 4.436009965197357e-07, CAM6: 2.1951985171352138e-07
var: Q; Level 125.0mb; RMSE NIense: 2.5196813605943166e-07, SUPERense: 2.693049973564704e-07, SUMO5: 1.8647682050476643e-07, SUMO6: 3.8075833395031466e-07, CAM5: 2.986627382577933e-07, CAM6: 2.462385471927077e-07
var: Q; Level 150.0mb; RMSE NIense: 5.177318106157034e-07, SUPERense: 4.860189710342592e-07, SUMO5: 5.324161722560269e-07, SUMO6: 5.762896819236335e-07, CAM5: 6.361503901186789e-07, CAM6: 5.505600974844282e-07
var: Q; Level 175.0mb; RMSE NIense: 1.5819371068489263e-06, SUPERense: 1.4855972025955836e-06, SUMO5: 1.6286604263918059e-06, SUMO6: 1.6175744840984277e-06, CAM5: 1.9058429408107636e-06, CAM6: 1.5760871825693052e-06
var: Q; Level 200.0mb; RMSE NIense: 3.3526567407684464e-06, SUPERense: 3.491843776266927e-06, SUMO5: 3.4180503106303

In [ ]:
%%time
DSsuper5 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/CAM5_Betzy1/climo/CAM5_Betzy1_PRECT_climo.nc')
DSsuper5_season = seas_mean(DSsuper5,df,SEAS)*86400000   # m/s --> mm/day

DSc5 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/MME_Regular/climo/MME_Regular_PRECT_climo.nc')
DSMME_season = seas_mean(DSc5,df,SEAS)*86400000

DSc5 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/FHIST5_f09_g16_v2/climo/FHIST5_f09_g16_v2_PRECT_climo.nc')
DSCAM5_season = seas_mean(DSc5,df,SEAS)*86400000

DSsuper6 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/CAM6_Betzy1/climo/CAM6_Betzy1_PRECT_climo.nc')
DSsuper6_season = seas_mean(DSsuper6,df,SEAS)*86400000

DSc6 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/FHIST_f09_g16/climo/FHIST_f09_g16_PRECT_climo.nc')
DSCAM6_season = seas_mean(DSc6,df,SEAS)*86400000

DSc6 = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/ADF/MME_Combined/climo/MME_Combined_PRECT_climo.nc')
DSMMEr_season = seas_mean(DSc6,df,SEAS)*86400000

DSobs = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/Reanalysis/GPCP/GPCP.cam6grid.monthly_climo.1997-2023.nc')
DSobs_season = seas_mean(DSobs,df,SEAS)
DSobs['PRECT'] = DSobs['precip']
DSobs_season['PRECT'] = DSobs_season['precip']  